In [29]:
import pandas as pd
from transformers import LongformerTokenizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences

# Read data
df = pd.read_csv(r"B:\project\chunks\chunk_2.csv")
essays = df['text'].tolist()

# Tokenize using LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokens = tokenizer(essays, return_tensors='pt', padding=True, truncation=True)
x = tokens['input_ids']
y = df['generated'].values

In [42]:
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)

Input IDs size: torch.Size([12180, 4096])
Attention Mask size: torch.Size([12180, 4096])


In [43]:
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)


Vocabulary Size: 26781


In [47]:
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Max Sequence Length: 4096


In [51]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

# Pad or truncate the input sequences
# max_sequence_length_x = max(numel(seq) for seq in xtrain[0])  # Assuming 'input_ids' is the first element in xtrain
input_length=4096
xtrain_padded = pad_sequences(xtrain, maxlen=input_length, padding='post')

# Build and compile the model
model = Sequential()
model.add(Embedding(input_dim=51000, output_dim=100, input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(xtrain_padded, ytrain, epochs=5, batch_size=32)

Epoch 1/5
305/305 [==============================] - 735s 2s/step - loss: 0.6760 - accuracy: 0.5955
Epoch 2/5
305/305 [==============================] - 838s 3s/step - loss: 0.6755 - accuracy: 0.5956
Epoch 3/5
305/305 [==============================] - 721s 2s/step - loss: 0.6750 - accuracy: 0.5956
Epoch 4/5
305/305 [==============================] - 626s 2s/step - loss: 0.6752 - accuracy: 0.5956
Epoch 5/5
305/305 [==============================] - 615s 2s/step - loss: 0.6754 - accuracy: 0.5956


In [52]:
model.save('model02.keras')

In [33]:
print(len(x))
print(len(y))

12180
12180


In [36]:
print(x.shape)
print(y.shape)

torch.Size([12180, 4096])
(12180,)


In [37]:
print(xtrain.shape)
print(ytrain.shape)

torch.Size([9744, 4096])
(9744,)
